# 🍏 Fun & Fit Health Advisor Agent Tutorial 🍎

Welcome to our **Fun & Fit Health Advisor Agent** tutorial, where you'll use **Azure AI Foundry** SDKs to create a playful (yet carefully disclaimed!) health and fitness assistant. We'll:

1. **Initialize** our project using Azure.AI.Projects.
2. **Create** an Agent specialized in providing general wellness and nutritional advice (with disclaimers!).
3. **Manage** conversations about fitness, nutrition, and general health topics.
4. **Showcase** logging and tracing with **OpenTelemetry**.
5. **Demonstrate** how to incorporate tools, safety disclaimers, and basic best practices.

## ⚠️ Important Medical Disclaimer ⚠️
> **The health information provided by this notebook is for general educational and entertainment purposes only and is not intended as a substitute for professional medical advice, diagnosis, or treatment**. Always seek the advice of your physician or other qualified health provider with any questions you may have regarding a medical condition. Never disregard professional medical advice or delay seeking it because of something you read or receive from this notebook.

## Prerequisites
Complete [the notebooks in introduction](../1.%20initial-setup/quick_start.ipynb)


## Let's Get Started

We'll walk you through each cell with notes and diagrams to keep it fun. Let's begin!

<img src="seq-diagrams/1-basics.png">

# 🔐 Authentication Setup

Before running the next cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

> az login --use-device-code

This will provide you with a device code and URL to authenticate in your browser, which is useful for:

- Remote development environments
- Systems without a default browser
- Corporate environments with strict security policies

After successful authentication, you can proceed with the notebook cells below.

# 1. Initial Setup

We'll start by importing needed libraries, loading environment variables, and initializing an AIProjectClient so we can do all the agent-related actions. Let's do it! 🎉

In [ ]:
#pragma warning disable OPENAI001

#r "nuget: Azure.Identity, 1.18.0-beta.2"
#r "nuget: Azure.AI.Projects, 1.2.0-beta.5"
#r "nuget: dotenv.net"

using System.IO;
using System.ClientModel.Primitives;
using Azure.Identity;
using Azure.AI.Projects;
using Azure.AI.Projects.OpenAI;
using OpenAI.Responses;
using dotenv.net;  

DotEnv.Load(new DotEnvOptions(envFilePaths: new[] { Path.Combine(".","..", ".env") })); 

In [ ]:
var tenantId = Environment.GetEnvironmentVariable("TENANT_ID");
var projectEndpoint = Environment.GetEnvironmentVariable("AI_FOUNDRY_PROJECT_ENDPOINT");
var modelDeployment = Environment.GetEnvironmentVariable("MODEL_DEPLOYMENT_NAME");
Console.WriteLine($"🔑 Using Tenant ID: {tenantId}");

AIProjectClient projectClient;

try
{
    var credential = new DefaultAzureCredential(new DefaultAzureCredentialOptions
    {
        TenantId = tenantId
    });

    projectClient = new AIProjectClient(new Uri(projectEndpoint), credential);
    Console.WriteLine("✅ Successfully initialized AIProjectClient");
}
catch (Exception ex)
{
    Console.WriteLine($"❌ Error initializing AIProjectClient: {ex.Message}");
    throw;
}

# 2. Creating our Fun & Fit Health Advisor Agent 🏋️

We'll create an Agent specialized in general health and wellness. We'll explicitly mention disclaimers in its instructions, so it never forgets to keep it safe! The instructions also ask the agent to focus on general fitness, dietary tips, and always encourage the user to seek professional advice.

In [ ]:
async Task<AgentVersion> CreateHealthAdvisorAgent()
{
    try
    {
        var modelName = Environment.GetEnvironmentVariable("MODEL_DEPLOYMENT_NAME"); 
        var agentDefinition = new PromptAgentDefinition(modelName)
        {
            Instructions = @"You are a friendly AI Health Advisor.
            You provide general health, fitness, and nutrition information, but always:
            1. Include medical disclaimers.
            2. Encourage the user to consult healthcare professionals.
            3. Provide general, non-diagnostic advice around wellness, diet, and fitness.
            4. Clearly remind them you're not a doctor.
            5. Encourage safe and balanced approaches to exercise and nutrition.
            "
        };
        AgentVersion agentVersion = await projectClient.Agents.CreateAgentVersionAsync(
            agentName:"fun-fit-health-advisor", 
            options: new(agentDefinition));
        Console.WriteLine($"Agent created (id: {agentVersion.Id}, name: {agentVersion.Name}, version: {agentVersion.Version})");
        return agentVersion;
    }
    catch (Exception ex)
    {
        Console.WriteLine($"❌ Error creating Health Advisor Agent: {ex.Message}");
        return null;
    }   
}

var healthAdvisorAgent = await CreateHealthAdvisorAgent();

# 3. Managing Our Health Conversations 💬

A conversation (or thread) is where we'll store the user's messages and the agent's responses about health topics. Let's create a new thread dedicated to Health & Fitness Q&A.

In [ ]:
async Task<ProjectConversation> StartHealthConversation()
{
    ProjectConversation conversation = await projectClient.OpenAI.Conversations.CreateProjectConversationAsync();
    return conversation;
} 
ProjectConversation healthConversation = await StartHealthConversation();
Console.WriteLine($"Started Health Conversation (id: {healthConversation.Id})");

# 4. Asking Health & Fitness Questions 🏃‍♂️

We'll create messages from the user about typical health questions. For example, **"How do I calculate my BMI?"** or **"What's a balanced meal for an active lifestyle?"**. We'll let our Health Advisor Agent respond, always remembering that disclaimer!

In [ ]:
#pragma warning disable OPENAI001
ProjectResponsesClient responsesClient = null;
async Task<ResponseResult> ChatWithHealthAgent(string userQuestion)
{
    Console.WriteLine($"👤 User: {userQuestion}");
    try
    {
    if(responsesClient == null)
    {
        responsesClient = projectClient.OpenAI.GetProjectResponsesClientForAgent(
            defaultAgent: healthAdvisorAgent,
            defaultConversationId: healthConversation.Id);
    }

    ResponseResult response = await responsesClient.CreateResponseAsync(userQuestion);
    Console.WriteLine("✅ Agent processing completed!");
    Console.WriteLine("🤖 Agent:");
    Console.WriteLine(response.GetOutputText());
    return response;
    }
    catch (Exception ex)
    {
        Console.WriteLine($"❌ Error during chat with Health Agent: {ex.Message}");
        return null;
    }
}

In [ ]:
#pragma warning disable OPENAI001
Console.WriteLine("🧪 Testing the Health & Fitness Agent...");

Console.WriteLine("Test 1: Simple Health Question ✅");

Console.WriteLine("Test 2: Workout Advice");
ResponseResult response1 = await ChatWithHealthAgent("What's a good 30-minute workout routine for a beginner who wants to build strength?");
Console.WriteLine();

Console.WriteLine("Test 3: Nutrition Advice");
ResponseResult response2 = await ChatWithHealthAgent("Can you suggest a balanced meal plan for someone aiming to increase muscle mass?");
Console.WriteLine();

Console.WriteLine("Test 4: Goal Setting");
ResponseResult response3 = await ChatWithHealthAgent("How can I set realistic fitness goals for someone who wants to lose 20 pounds safely?");

Console.WriteLine();
int successfulTests = new object[] { true, response1, response2, response3 }.Count(r => r != null);
Console.WriteLine($"📊 Summary: {successfulTests}/4 tests passed successfully");


# 5. Final Test: Complex Health Question 🧹

In [ ]:
#pragma warning disable OPENAI001
// Displat Conversation History given a conversation ID
async Task ShowConversationHistory(string conversationId)
{
    if(string.IsNullOrEmpty(conversationId))
    {
        Console.WriteLine("❌ No Conversation ID provided.");
        return;
    }
    try
    {

    Console.WriteLine($"📝 Conversation History for ID: {conversationId}");
    var itemCount = 0;
    await foreach (AgentResponseItem item in projectClient.OpenAI.Conversations.GetProjectConversationItemsAsync(
    healthConversation.Id, order:"asc"))
    {
        itemCount++;
        var messageResponseItem = item.AsResponseResultItem() as MessageResponseItem;
        var role = messageResponseItem.Role.ToString();
        var roleEmoji = role == "User" ? "👤" : "🤖";
        Console.WriteLine($"{itemCount}. {roleEmoji} {role}:");
        Console.WriteLine(new string('-', 40));
        var content = messageResponseItem.Content?[0]?.Text;
        if(content != null && content.Length > 0)
        {
            if(content.Length > 500)
            {
                Console.WriteLine($"{content.Substring(0,500)}... [truncated]");
            } 
            else
            {
                Console.WriteLine(content);
            }
        }
        else
        {
            Console.WriteLine("No content available.");
        }
    }
    Console.WriteLine($"\n📊 Total messages in conversation: {itemCount}");
    }
    catch(Exception ex)
    {
        Console.WriteLine($"❌ Error retrieving conversation history: {ex.Message}");
    }
}

await ShowConversationHistory(healthConversation.Id);
Console.WriteLine(new string('=', 40));

In [ ]:

Console.WriteLine("🔍 Final Test: Complex Health Question");
Console.WriteLine(new string('=', 40));

var finalResponse = await ChatWithHealthAgent("I'm a 35-year-old office worker who sits most of the day. I want to start exercising but I have limited time (only 20 minutes, 3 times per week). What would you recommend for someone with back pain from sitting too long?");

if (finalResponse != null)
{
    Console.WriteLine("\n✅ Final test successful!");
    Console.WriteLine("\n🎯 Our Health & Fitness Agent is fully functional and ready to help users with personalized advice!");
}
else
{
    Console.WriteLine("\n❌ Final test failed.");
}

# 5. Cleanup 🧹

If you'd like to remove your agent from the service once finished, you can do so below. (In production, you might keep your agent around for stateful experiences!)

In [ ]:
// Clean up resources
// await projectClient.OpenAI.Conversations.DeleteConversationAsync(healthConversation.Id);
// await projectClient.Agents.DeleteAgentAsync(healthAdvisorAgent.Id);
Console.WriteLine("🗑️ Deleted agent");
Console.WriteLine("✅ Cleanup completed!");

# Congratulations! 🏆

You've successfully built a **Fun & Fit Health Advisor** that can:

1. **Respond** to basic health and fitness questions.
2. **Use disclaimers** to encourage safe, professional consultation.
3. **Provide** general diet and wellness information.
4. **Use** the synergy of **Azure AI Foundry** modules to power the conversation.

## Next Steps
- Explore adding more advanced tools (like **FileSearchTool** or **CodeInterpreterTool**) to provide more specialized info.
- Evaluate your AI's performance with **Azure AI Evaluation**!
- Add **OpenTelemetry** or **Azure Monitor** for deeper insights.
- Incorporate **function calling** if you want to handle things like advanced calculation or direct data analysis.

**Let's proceed to [2-code_interpreter.ipynb](./2-code_interpreter.ipynb)**

Happy (healthy) coding! 💪

# 🎯 Summary & Next Steps

Congratulations! You've successfully:

✅ Connected to Azure AI Foundry - Established secure connection using authentication <br>
✅ Created an AI Agent - Built a specialized Health & Fitness Assistant <br>
✅ Implemented Chat Functionality - Created functions to interact with the agent <br>
✅ Tested Multiple Scenarios - Verified the agent works for various health questions <br>

### Key Concepts Learned
- **Azure AI Agent Service** - A managed service for building intelligent AI assistants
- **Conversations** - Conversation sessions that maintain context between messages
- **Responses** - Individual communications between users and agents

### What's Next?

This was just the beginning! In the next notebooks, you'll learn about:

- Code Interpreter - Agents that can write and execute code
- File Search - Agents that can search through documents
- Bing Grounding - Agents with real-time web search capabilities
- AI Search Integration - Agents connected to your knowledge bases
- Azure Functions - Agents that can trigger actions and workflows

### 🔧 Key Takeaways
- The **CreateResponse** method is the simplest way to interact with agents
- Agents automatically maintain conversation context within Conversation objects
- The agent provides responsible AI disclaimers for health advice
- Error handling is crucial when working with AI services

Ready to explore more advanced agent capabilities? Continue with the next notebook! 🚀